# Mars Facts Scrape

In [39]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np
import csv
import json

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
# Determining number of years in the website
html = '''
<form class="year-select" action="/vehicle-registration" accept-charset="UTF-8" method="get">
  <label for="year" aria-hidden="false" style="display: none;">Year</label>
  <select id="year" name="year" onchange="this.form.submit()" class="form-control year">
    <option selected="selected" value="2021">2021</option>
<option value="2020">2020</option>
<option value="2019">2019</option>
<option value="2018">2018</option>
<option value="2017">2017</option>
<option value="2016">2016</option>
  </select>
</form>
'''
items = soup(html, 'lxml')
options = items.find_all('option')
year =[]
for item in options:
    year.append(int(item.text.strip()))


In [46]:
counter = 1
for item in year:
    # Scrapping data for available years
    url = f'https://afdc.energy.gov/vehicle-registration?year={item}'
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, 'html.parser')
    table = html_soup.find('table', class_='table table-bordered table-striped vehicle-registration dataTable')
    car_registeration = []
    rows = table.find_all('tr')
    # Skipping the first two empty rows
    for row in rows[2:]:
        row_data = row.find_all('td')
        data_sub =[]
        for items in row_data:
            data_sub.append(items.text.strip())
        car_registeration.append(data_sub)
    car_registeration_pd = pd.DataFrame(car_registeration)
    
    car_registeration_pd.columns=['State', 'EV', 'PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
    dflist = ['EV', 'PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
#     car_registeration_pd = car_registeration_pd.set_index(car_registeration_pd.columns[0])
    # Finding the total number of cars sold in every state
    for column in dflist:
        car_registeration_pd[f'{column}']= car_registeration_pd[f'{column}'].str.replace(',', '').astype(float)
    car_registeration_pd['Total Sales'] = car_registeration_pd[dflist].sum(axis = 1)
    car_registeration_pd['EV Share'] = 100 * car_registeration_pd['EV']/car_registeration_pd['Total Sales']
    columns_to_be_removed = ['PHEV', 'HEV', 'BIODIESEL', 'E85', 'CNG', 'PROPANE', 'HYDROGEN', 'METHANOL', 'GASOLINE', 'DIESEL']
    car_registeration_pd.drop(columns_to_be_removed, inplace = True, axis =1)
    car_registeration_pd.rename(columns = {'EV': f'EV_{item}', 'Total Sales': f'Total Sales_{item}', 'EV Share': f'EV Share_{item}'}, inplace = True)
#     car_registeration_pd.to_csv(f'{item}.csv')
    if counter == 1:
        car_registeration_pd_total = pd.DataFrame(car_registeration_pd)
        counter = counter + 1
    else:
        car_registeration_pd_total = pd.merge(car_registeration_pd_total, car_registeration_pd, on='State', how='inner')
car_registeration_pd_total = car_registeration_pd_total.set_index(car_registeration_pd.columns[0])
car_registeration_pd_total.head()

car_registeration_pd_total.to_csv('data.csv')
    
    
    
    

In [50]:
def make_json(csv_file_path, json_file_path):
    data = {}
    with open(csv_file_path, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        i = 1
        for rows in csvReader:
#             key = rows['State']
            data [i]=rows
            i = i+1
    with open (json_file_path, 'w', encoding = 'utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
csv_file_path = r'data.csv'
json_file_path = r'data.json'

make_json(csv_file_path, json_file_path)

In [15]:
# Quit the browsing session
browser.quit()